##### Case Study - Whats Cooking (Kaggle Dataset)

Import Libararies 

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
import re
import nltk
import random

from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore")

Read Datafiles 

In [3]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

#### Exploratory Data Analysis 

In [5]:
print(train.head(2))
print(test.head(2))

       cuisine     id                                        ingredients
0        greek  10259  [romaine lettuce, black olives, grape tomatoes...
1  southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...
      id                                        ingredients
0  18009  [baking powder, eggs, all-purpose flour, raisi...
1  28583  [sugar, egg yolks, corn starch, cream of tarta...


In [6]:
# Removing space within words from the ingredients column 
new = []
for items in train.ingredients:
    items = [item.replace(' ', '') for item in items]
    new.append(items)

In [7]:
new_test = []
for items in test.ingredients:
    items = [item.replace(' ', '') for item in items]
    new_test.append(items)

In [8]:
# Removing List of Lists and creating additional column with new ingredient list 
print(new[0:10])
new_transposed = zip(new)
new = pd.DataFrame(new_transposed)
print(new.head())

[['romainelettuce', 'blackolives', 'grapetomatoes', 'garlic', 'pepper', 'purpleonion', 'seasoning', 'garbanzobeans', 'fetacheesecrumbles'], ['plainflour', 'groundpepper', 'salt', 'tomatoes', 'groundblackpepper', 'thyme', 'eggs', 'greentomatoes', 'yellowcornmeal', 'milk', 'vegetableoil'], ['eggs', 'pepper', 'salt', 'mayonaise', 'cookingoil', 'greenchilies', 'grilledchickenbreasts', 'garlicpowder', 'yellowonion', 'soysauce', 'butter', 'chickenlivers'], ['water', 'vegetableoil', 'wheat', 'salt'], ['blackpepper', 'shallots', 'cornflour', 'cayennepepper', 'onions', 'garlicpaste', 'milk', 'butter', 'salt', 'lemonjuice', 'water', 'chilipowder', 'passata', 'oil', 'groundcumin', 'bonelesschickenskinlessthigh', 'garammasala', 'doublecream', 'naturalyogurt', 'bayleaf'], ['plainflour', 'sugar', 'butter', 'eggs', 'freshgingerroot', 'salt', 'groundcinnamon', 'milk', 'vanillaextract', 'groundginger', 'powderedsugar', 'bakingpowder'], ['oliveoil', 'salt', 'mediumshrimp', 'pepper', 'garlic', 'choppedci

In [9]:
print(new_test[0:10])
new_test_transposed = zip(new_test)
new_test = pd.DataFrame(new_test_transposed)
print(new_test.head())

[['bakingpowder', 'eggs', 'all-purposeflour', 'raisins', 'milk', 'whitesugar'], ['sugar', 'eggyolks', 'cornstarch', 'creamoftartar', 'bananas', 'vanillawafers', 'milk', 'vanillaextract', 'toastedpecans', 'eggwhites', 'lightrum'], ['sausagelinks', 'fennelbulb', 'fronds', 'oliveoil', 'cubanpeppers', 'onions'], ['meatcuts', 'filepowder', 'smokedsausage', 'okra', 'shrimp', 'andouillesausage', 'water', 'paprika', 'hotsauce', 'garliccloves', 'browning', 'lumpcrabmeat', 'vegetableoil', 'all-purposeflour', 'freshlygroundpepper', 'flatleafparsley', 'bonelesschickenskinlessthigh', 'driedthyme', 'whiterice', 'yellowonion', 'ham'], ['groundblackpepper', 'salt', 'sausagecasings', 'leeks', 'parmigianoreggianocheese', 'cornmeal', 'water', 'extra-virginoliveoil'], ['bakingpowder', 'all-purposeflour', 'peachslices', 'cornstarch', 'heavycream', 'lemonjuice', 'unsaltedbutter', 'salt', 'whitesugar'], ['grapejuice', 'orange', 'whitezinfandel'], ['groundginger', 'whitepepper', 'greenonions', 'orangejuice', 

In [10]:
# conacate additional column with train
train = pd.concat([train,new],axis = 1)

In [11]:
test = pd.concat([test,new_test],axis = 1)

In [12]:
# Renaming new column 
train = train.rename(columns={0: "ingredients_2"})

In [13]:
test = test.rename(columns={0: "ingredients_2"})

In [14]:
train.head(2)

,cuisine,id,ingredients,ingredients_2
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...","[romainelettuce, blackolives, grapetomatoes, g..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...","[plainflour, groundpepper, salt, tomatoes, gro..."


In [15]:
test.head(2)

,id,ingredients,ingredients_2
0,18009,"[baking powder, eggs, all-purpose flour, raisi...","[bakingpowder, eggs, all-purposeflour, raisins..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta...","[sugar, eggyolks, cornstarch, creamoftartar, b..."


In [16]:
# Removing commas (',' ) from ingredient2 and saving in data in additional column 
train['ingredients_1'] =train['ingredients_2'].apply(' '.join)

In [17]:
test['ingredients_1'] =test['ingredients_2'].apply(' '.join)

In [18]:
train.head()

,cuisine,id,ingredients,ingredients_2,ingredients_1
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...","[romainelettuce, blackolives, grapetomatoes, g...",romainelettuce blackolives grapetomatoes garli...
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...","[plainflour, groundpepper, salt, tomatoes, gro...",plainflour groundpepper salt tomatoes groundbl...
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...","[eggs, pepper, salt, mayonaise, cookingoil, gr...",eggs pepper salt mayonaise cookingoil greenchi...
3,indian,22213,"[water, vegetable oil, wheat, salt]","[water, vegetableoil, wheat, salt]",water vegetableoil wheat salt
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...","[blackpepper, shallots, cornflour, cayennepepp...",blackpepper shallots cornflour cayennepepper o...


In [17]:
test.head(2)

,id,ingredients,ingredients_2,ingredients_1
0,18009,"[baking powder, eggs, all-purpose flour, raisi...","[bakingpowder, eggs, all-purposeflour, raisins...",bakingpowder eggs all-purposeflour raisins mil...
1,28583,"[sugar, egg yolks, corn starch, cream of tarta...","[sugar, eggyolks, cornstarch, creamoftartar, b...",sugar eggyolks cornstarch creamoftartar banana...


In [19]:
# Train-Test Split 
train_corpus = train['ingredients_1']
test_corpus = test['ingredients_1']
y = train['cuisine']
X  = train_corpus

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [20]:
print(train_corpus.shape)
print(test_corpus.shape)
print(X_train.shape)
print(X_test.shape)

(39774,)
(9944,)
(27841,)
(11933,)


In [20]:
# Remove array
#train['ingredients_clean_string'] = [' , '.join(z).strip() for z in train['ingredients']]   
#test['ingredients_clean_string'] = [' , '.join(z).strip() for z in test['ingredients']]

#### Creating TFIDF vector metrics 

In [21]:
train_vectorizer = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),analyzer="word", 
                             max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)

In [22]:
train_tfidf=train_vectorizer.fit_transform(X_train).todense()

In [23]:
train_tfidf.shape

(27841, 6166)

In [24]:
val_tfidf = train_vectorizer.transform(X_test)

In [25]:
val_tfidf.shape

(11933, 6166)

In [26]:
test_tfidf=train_vectorizer.transform(test_corpus)

In [27]:
test_tfidf.shape

(9944, 6166)

In [28]:
train_tfidf

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.29395641, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [29]:
val_tfidf

<11933x6166 sparse matrix of type '<class 'numpy.float64'>'
	with 130091 stored elements in Compressed Sparse Row format>

In [30]:
test_tfidf

<9944x6166 sparse matrix of type '<class 'numpy.float64'>'
	with 108817 stored elements in Compressed Sparse Row format>

#### Model - 1 SVC

In [31]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
model = LinearSVC(C=0.80, penalty="l2", dual=False)
parameters = {'C':[1, 10]}
classifier = GridSearchCV(model, parameters)
classifier.fit(train_tfidf,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=LinearSVC(C=0.8, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=None, param_grid={'C': [1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [32]:
train_pred = classifier.predict(train_tfidf)
val_pred = classifier.predict(val_tfidf)
test_pred = classifier.predict(test_tfidf)

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_train, train_pred))
print(classification_report(y_test, val_pred))

              precision    recall  f1-score   support

   brazilian       0.94      0.86      0.90       327
     british       0.89      0.78      0.83       578
cajun_creole       0.90      0.87      0.89      1046
     chinese       0.91      0.94      0.93      1889
    filipino       0.94      0.89      0.91       522
      french       0.82      0.81      0.82      1834
       greek       0.93      0.87      0.90       827
      indian       0.93      0.97      0.95      2107
       irish       0.91      0.82      0.87       477
     italian       0.90      0.96      0.93      5470
    jamaican       0.96      0.91      0.94       369
    japanese       0.95      0.85      0.90       989
      korean       0.97      0.93      0.95       595
     mexican       0.96      0.97      0.96      4531
    moroccan       0.94      0.93      0.93       569
     russian       0.89      0.80      0.85       335
 southern_us       0.87      0.92      0.89      3015
     spanish       0.88    

In [34]:
test_predict = pd.DataFrame(test_pred)
sub = pd.concat([test['id'],test_predict],axis=1)

In [35]:
sub.to_csv("svc.csv",index=True)

###### Model 2: Random Forest

In [37]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(train_tfidf, y_train)
train_pred_rf = rf.predict(train_tfidf)
val_pred_rf = rf.predict(val_tfidf)
test_pred_rf = rf.predict(test_tfidf)

In [38]:
print(classification_report(y_train, train_pred_rf))
print(classification_report(y_test, val_pred_rf))

              precision    recall  f1-score   support

   brazilian       1.00      1.00      1.00       327
     british       1.00      1.00      1.00       578
cajun_creole       1.00      1.00      1.00      1046
     chinese       1.00      1.00      1.00      1889
    filipino       1.00      1.00      1.00       522
      french       1.00      1.00      1.00      1834
       greek       1.00      1.00      1.00       827
      indian       1.00      1.00      1.00      2107
       irish       1.00      1.00      1.00       477
     italian       1.00      1.00      1.00      5470
    jamaican       1.00      1.00      1.00       369
    japanese       1.00      1.00      1.00       989
      korean       1.00      1.00      1.00       595
     mexican       1.00      1.00      1.00      4531
    moroccan       1.00      1.00      1.00       569
     russian       1.00      1.00      1.00       335
 southern_us       1.00      1.00      1.00      3015
     spanish       1.00    

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
rfc = RandomForestClassifier(n_jobs=-1, max_features='sqrt') 
 
#Use a grid over parameters of interest
param_grid = { 
           "n_estimators" : [300, 500, 700],
           "max_depth" : [None, 1, 2, 3],
           "min_samples_leaf" : [1, 2, 3]}

In [ ]:
clf = GridSearchCV(estimator=rfc, param_grid=param_grid,cv=5)
clf.fit(train_tfidf, y_train)

In [ ]:
print(clf.best_params_)

In [ ]:
rfc_best = RandomForestClassifier(**clf.best_params_)
rfc_best.fit(train_tfidf, y_train)
train_pred_rfc_best = rfc_best.predict(train_tfidf)
val_pred_rfc_best = rfc_best.predict(val_tfidf)
test_pred_rfc_best = rfc_best.predict(test_tfidf)

In [45]:
print(classification_report(y_train, train_pred_rf))
print(classification_report(y_test, val_pred_rf))

              precision    recall  f1-score   support

   brazilian       1.00      1.00      1.00       320
     british       1.00      1.00      1.00       583
cajun_creole       1.00      1.00      1.00      1059
     chinese       1.00      1.00      1.00      1846
    filipino       1.00      1.00      1.00       525
      french       1.00      1.00      1.00      1897
       greek       1.00      1.00      1.00       828
      indian       1.00      1.00      1.00      2096
       irish       1.00      1.00      1.00       476
     italian       1.00      1.00      1.00      5479
    jamaican       1.00      1.00      1.00       357
    japanese       1.00      1.00      1.00       968
      korean       1.00      1.00      1.00       574
     mexican       1.00      1.00      1.00      4543
    moroccan       1.00      1.00      1.00       578
     russian       1.00      1.00      1.00       337
 southern_us       1.00      1.00      1.00      3059
     spanish       1.00    

In [47]:
test_predict_rf = pd.DataFrame(test_pred_rf)
sub = pd.concat([test['id'],test_predict_rf],axis=1)

In [48]:
sub.to_csv("rf.csv",index=True)

###### Model 3: Stochastic Gradient Descent 

In [41]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(shuffle=True, random_state=101)
sgd.fit(train_tfidf,y_train)
train_pred_sgd = sgd.predict(train_tfidf)
val_pred_sgd = sgd.predict(val_tfidf)
test_pred_sgd = sgd.predict(test_tfidf)

In [42]:
print(classification_report(y_train, train_pred_sgd))
print(classification_report(y_test, val_pred_sgd))

              precision    recall  f1-score   support

   brazilian       0.88      0.53      0.66       320
     british       0.90      0.44      0.59       583
cajun_creole       0.86      0.76      0.81      1059
     chinese       0.84      0.90      0.87      1846
    filipino       0.89      0.71      0.79       525
      french       0.77      0.64      0.70      1897
       greek       0.84      0.76      0.80       828
      indian       0.87      0.94      0.90      2096
       irish       0.86      0.50      0.63       476
     italian       0.77      0.95      0.85      5479
    jamaican       0.85      0.76      0.80       357
    japanese       0.93      0.72      0.81       968
      korean       0.92      0.80      0.86       574
     mexican       0.91      0.94      0.92      4543
    moroccan       0.91      0.76      0.83       578
     russian       0.87      0.52      0.65       337
 southern_us       0.74      0.87      0.80      3059
     spanish       0.88    

In [43]:
test_predict_sgd = pd.DataFrame(test_pred_sgd)
sub = pd.concat([test['id'],test_predict_sgd],axis=1)

In [44]:
sub.to_csv("sgd.csv",index=True)